In [ ]:
import numpy as np
from scipy.linalg import solve, schur
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import LinearOperator
from utility import orbit, BrusselatorModel, optim_BrusselatorModel
import matplotlib.pyplot as plt
from time import time

In [ ]:
if __name__ == "__main__":
    param_file = "./brusselator_params_2.in"  # JSON file containing model parameters
    model = BrusselatorModel(param_file)
    model_optim = optim_BrusselatorModel(param_file)
    Max_iter = 1
    epsilon = 1e-6
    steps = 80
    Time_full_jac = np.zeros((6))
    k = 0
    for n_z in [16,32,64,128,256,512]:
        model.n_z = n_z
        I = np.eye(2 * (n_z - 2))
        print(f"Number of grid points: {n_z}")
        model.Lap = model.Lap_mat(n_z)
        f = model.dydt
        Jacf = model.brusselator_jacobian

            
        X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
        Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
        y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
        
        orbit_finder = orbit(f,y0,model.T_ini, Jacf,2,solve_ivp, 'RK45',steps,Max_iter, epsilon)

        start = time()
        orbit_finder.integ_monodromy(y0=y0,M0=I,T=model.T_ini)
        end = time()
        Time_full_jac[k] = end - start
        print(f"Time for integration: {Time_full_jac[k]} seconds")
        k += 1

In [ ]:
p0, pe = 5 ,4
subspace_iter = 5
rho = 0.5

Time_NP_full_jac = np.zeros((6))
k = 0
for n_z in [16,32,64,128,256,512]:
    model.n_z = n_z
    I = np.eye(2 * (n_z - 2))
    print(f"Number of grid points: {n_z}")
    model.Lap = model.Lap_mat(n_z)
    f = model.dydt
    Jacf = model.brusselator_jacobian

        
    X0 = model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
    Y0 = model.B/model.A + 0.1*np.sin(np.pi*(np.linspace(0, model.z_L, model.n_z)/model.z_L))
    y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
    

    V_0 = np.eye(len(y0))[:,:p0+pe]
    v0 = V_0[:,0]
    orbit_finder = orbit(f,y0,model.T_ini, Jacf,2,solve_ivp, 'RK45',steps,Max_iter, epsilon)

    MV = LinearOperator((len(y0),len(y0)),matvec = lambda v : orbit_finder.monodromy_mult_matvec(y0,model.T_ini, v, method=2, epsilon=1e-6),
                             matmat = lambda V : orbit_finder.monodromy_mult(y0,model.T_ini, V, method=2, epsilon=1e-6))
    start = time()
    for i in range(subspace_iter):
        MV @ V_0 #Total integration in the subspace iteration 
        
    MV @ v0 #Integration in the Picard correction step
    MV @ V_0[:,p0:] #Integration in the Newton correction step
    #For the rhs approximation in the Newton correction step
    sol = solve_ivp(fun=f,t_span=[0.0, model.T_ini],
           t_eval= [model.T_ini], 
            y0=y0, method='RK45', 
            **{"rtol": 1e-7,"atol":1e-9, "jac": Jacf}
            )

    end = time()
    Time_NP_full_jac[k] = end - start
    print(f"Time for integration: {Time_NP_full_jac[k]} seconds")
    k += 1

In [ ]:
# np.save('./Results/ivp_solve_time/Newton_ivp_solve_time_full_jac',Time_full_jac)
# np.save('./Results/ivp_solve_time/NP_ivp_solve_time_full_jac',Time_NP_full_jac)
# np.save('./Results/ivp_solve_time/NP_ivp_solve_time_full_jac',Time_NP_sparse_jac)

Time_full_jac = np.load('./Results/ivp_solve_time/Newton_ivp_solve_time_full_jac.npy')
Time_NP_full_jac = np.load('./Results/ivp_solve_time/NP_ivp_solve_time_full_jac.npy')
Time_NP_sparse_jac = np.load('./Results/ivp_solve_time/NP_ivp_solve_time_sparse_jac.npy')
Time_sparse_jac = np.load('./Results/ivp_solve_time/Newton_ivp_solve_time_sparse_jac.npy')

In [ ]:
NZ = [16,32,64,128,256,512]
plt.figure(figsize=(10, 6))
# plt.plot(NZ, Time_full_jac,'o-', label='Newton dense jac', color='blue')
# plt.plot(NZ, Time_NP_full_jac, marker='x', label='Newton-Picard dense jac', color='red')
plt.plot(NZ, Time_NP_sparse_jac, marker='x', label='Newton-Picard sparse jac', color='orange')
plt.plot(NZ, Time_sparse_jac, marker='o', label='Newton sparse jac', color='purple')

# plt.plot(NZ, Time_NP_sparse_jac/Time_sparse_jac, marker='s', label='Newton/NP', color='green')
# plt.xscale('log')
plt.xlim(10, 1024)
# plt.yscale('log')
plt.xlabel('Number of grid points (n_z)')
plt.ylabel('Time (seconds)')
plt.title('Integration Time Comparison, and RK45 method')
plt.legend()
plt.grid(True)
# plt.savefig('./Results/ivp_solve_time/brusselator_ivpsolve_time_comparison.png', dpi=300)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

# Function defining the equilibrium condition
def F(y, lam):
    return lam * y - y**3

# Derivative wrt y
def Fy(y, lam):
    return lam - 3*y**2

# Compute equilibria for a range of lambda values
lambdas = np.linspace(-2, 2, 400)
branches = {'stable': [], 'unstable': [], 'lambda': []}

for lam in lambdas:
    # Always one real root at y=0
    branches['lambda'].append(lam)
    branches['unstable'].append(0.0)  # y=0

    if lam > 0:
        y_stable = np.sqrt(lam)
        branches['stable'].append(y_stable)
    else:
        branches['stable'].append(np.nan)  # No real branch here

# Mirror branch for symmetry
branches['mirror'] = [-y if not np.isnan(y) else np.nan for y in branches['stable']]

# Plotting
plt.figure(figsize=(8,6))
plt.plot(branches['lambda'], branches['stable'], label='Stable (+√λ)', color='green')
plt.plot(branches['lambda'], branches['unstable'], label='Unstable (0)', color='red')
plt.plot(branches['lambda'], branches['mirror'], label='Stable (–√λ)', color='green', linestyle='--')
plt.axvline(0, color='black', linestyle=':')
plt.xlabel('λ')
plt.ylabel('Equilibrium y')
plt.title('Branch of Equilibria for dy/dt = λy - y³')
plt.legend()
plt.grid(True)
plt.show()
